In [ ]:
# !git clone https://github.com/giorgiodemarchi/ImageBind.git

In [ ]:
# !git clone https://github.com/giorgiodemarchi/Visually-Indicated-Sounds.git  ## This was done in a external colab notebook

In [11]:
# !pip install boto3

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 139.3/139.3 kB 4.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.1/12.1 MB 53.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 82.2/82.2 kB 11.6 MB/s eta 0:00:00


In [ ]:
# !cd ImageBind; pip install .

In [1]:
from imagebind import data
import torch
import pandas as pd

from imagebind.models import imagebind_model
from imagebind.models.imagebind_model import ModalityType

from imagebind.data import transform_and_sample_video_tensor

/usr/local/lib/python3.10/dist-packages/torchvision/transforms/_functional_video.py:6: UserWarning: The 'torchvision.transforms._functional_video' module is deprecated since 0.12 and will be removed in the future. Please use the 'torchvision.transforms.functional' module instead.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torchvision/transforms/_transforms_video.py:22: UserWarning: The 'torchvision.transforms._transforms_video' module is deprecated since 0.12 and will be removed in the future. Please use the 'torchvision.transforms' module instead.
  warnings.warn(


In [2]:
device = "cuda:0" if torch.cuda.is_available() else "cpu"

In [8]:
# Instantiate model
model = imagebind_model.imagebind_huge(pretrained=True)
model.eval()

model.to(device)

ImageBindModel(
  (modality_preprocessors): ModuleDict(
    (vision): RGBDTPreprocessor(
      (cls_token): tensor((1, 1, 1280), requires_grad=True)
      
      (rgbt_stem): PatchEmbedGeneric(
        (proj): Sequential(
          (0): PadIm2Video()
          (1): Conv3d(3, 1280, kernel_size=(2, 14, 14), stride=(2, 14, 14), bias=False)
        )
      )
      (pos_embedding_helper): SpatioTemporalPosEmbeddingHelper(
        (pos_embed): tensor((1, 257, 1280), requires_grad=True)
        
      )
    )
    (text): TextPreprocessor(
      (pos_embed): tensor((1, 77, 1024), requires_grad=True)
      (mask): tensor((77, 77), requires_grad=False)
      
      (token_embedding): Embedding(49408, 1024)
    )
    (audio): AudioPreprocessor(
      (cls_token): tensor((1, 1, 768), requires_grad=True)
      
      (rgbt_stem): PatchEmbedGeneric(
        (proj): Conv2d(1, 768, kernel_size=(16, 16), stride=(10, 10), bias=False)
        (norm_layer): LayerNorm((768,), eps=1e-05, elementwise_affine=

**Create Dataset Instance and process video**

In [3]:
#from VisuallyIndicatedSounds.utils.datasets import StronglyLabelledDataset
from utils.datasets import StronglyLabelledDataset

# Ontology audioset dataset
ontology = pd.read_json('data/ontology.json')[['id','name']]

dataset = StronglyLabelledDataset()

In [5]:
video, audio, labels_df, info = dataset[2000]

video_fps, audio_fps = info['video_fps'], info['audio_fps']

print(f"Video Shape: {video.shape}")
print(f"Audio Shape: {audio.shape}")

labels_df = pd.merge(labels_df, ontology, how='left', left_on='label', right_on='id')

labels_df

/usr/local/lib/python3.10/dist-packages/torchvision/io/video.py:162: UserWarning: The pts_unit 'pts' gives wrong results. Please use pts_unit 'sec'.
  warnings.warn("The pts_unit 'pts' gives wrong results. Please use pts_unit 'sec'.")


Video Shape: torch.Size([250, 360, 480, 3])
Audio Shape: torch.Size([2, 441344])


,segment_id,start_time_seconds,end_time_seconds,label,id,name
0,3qDzHyrsWeg_30000,0.000,1.648,/m/07q2z82,/m/07q2z82,"Accelerating, revving, vroom"
1,3qDzHyrsWeg_30000,0.000,4.497,/m/03m9d0z,/m/03m9d0z,Wind
2,3qDzHyrsWeg_30000,0.000,4.497,/m/0838f,/m/0838f,Water
3,3qDzHyrsWeg_30000,0.000,4.511,/m/02rlv9,/m/02rlv9,"Motorboat, speedboat"
4,3qDzHyrsWeg_30000,4.623,10.000,/m/02rlv9,/m/02rlv9,"Motorboat, speedboat"
5,3qDzHyrsWeg_30000,4.623,10.000,/m/03m9d0z,/m/03m9d0z,Wind
6,3qDzHyrsWeg_30000,4.623,10.000,/m/0838f,/m/0838f,Water
7,3qDzHyrsWeg_30000,4.623,10.000,/m/07q2z82,/m/07q2z82,"Accelerating, revving, vroom"


In [6]:
# Visualize video
import torch
import numpy as np
from moviepy.editor import ImageSequenceClip

video_tensor = video
if isinstance(video_tensor, torch.Tensor):
    video_tensor = video_tensor.permute(0, 2, 1, 3).numpy()  # Reorder to (frames, height, width, channels)

total_frames = video_tensor.shape[0]
duration_in_seconds = 10 
fps = total_frames / duration_in_seconds

clip = ImageSequenceClip(list(video_tensor), fps=video_fps)
clip.ipython_display()

Moviepy - Building video __temp__.mp4.
Moviepy - Writing video __temp__.mp4



Moviepy - Done !
Moviepy - video ready __temp__.mp4


**Embed video**

In [10]:
## Working with the example video
## Required Transformations (Overwriting imagebind.data.load_and_transform_video)
scaled_video = transform_and_sample_video_tensor(video, device, clips_per_video=1)

print(f"Video Shape: {scaled_video.shape}")

inputs = {
    ModalityType.VISION: scaled_video,
}

with torch.no_grad():
    outputs = model(inputs)

print(f"Output Shape: {outputs['vision'].shape}")
video_embeddings = outputs['vision']

Video Shape: torch.Size([1, 250, 3, 224, 224])
Output Shape: torch.Size([1, 1024])


In [11]:
video_embeddings

tensor([[ 0.0303, -0.0129, -0.0057,  ..., -0.0206, -0.0081,  0.0164]],
       device='cuda:0')

In [12]:
ontology.name.unique()

array(['Human sounds', 'Human voice', 'Speech',
       'Male speech, man speaking', 'Female speech, woman speaking',
       'Child speech, kid speaking', 'Conversation',
       'Narration, monologue', 'Babbling', 'Speech synthesizer', 'Shout',
       'Bellow', 'Whoop', 'Yell', 'Battle cry', 'Children shouting',
       'Screaming', 'Whispering', 'Laughter', 'Baby laughter', 'Giggle',
       'Snicker', 'Belly laugh', 'Chuckle, chortle', 'Crying, sobbing',
       'Baby cry, infant cry', 'Whimper', 'Wail, moan', 'Sigh', 'Singing',
       'Choir', 'Yodeling', 'Chant', 'Mantra', 'Male singing',
       'Female singing', 'Child singing', 'Synthetic singing', 'Rapping',
       'Humming', 'Groan', 'Grunt', 'Yawn', 'Whistling', 'Wolf-whistling',
       'Respiratory sounds', 'Breathing', 'Wheeze', 'Snoring', 'Gasp',
       'Pant', 'Snort', 'Cough', 'Throat clearing', 'Sneeze', 'Sniff',
       'Human locomotion', 'Run', 'Shuffle', 'Walk, footsteps',
       'Digestive', 'Chewing, mastication', 'Biti

**Embed Text audio labels**

In [13]:
labels_list = ontology.name.unique()

In [14]:
from imagebind import data

inputs = {
    ModalityType.TEXT: data.load_and_transform_text(labels_list, device)
}

In [15]:
with torch.no_grad():
    output = model(inputs)

text_embeddings = output[ModalityType.TEXT]

In [16]:
text_embeddings.shape

torch.Size([632, 1024])

In [17]:
vision_text_similarity = torch.softmax(video_embeddings @ text_embeddings.T, dim=-1)

In [18]:
ontology['similarity'] = vision_text_similarity.cpu().numpy()[0]

In [19]:
ontology.sort_values('similarity', ascending=False).head()

,id,name,similarity
510,/m/07pc8l3,Snap,0.274083
349,/m/06mb1,Rain,0.109932
484,/m/0dv5r,Camera,0.091299
506,/m/083vt,Wood,0.064270
631,/m/04zc0,MP3,0.049526


In [20]:
labels_df

,segment_id,start_time_seconds,end_time_seconds,label,id,name
0,3qDzHyrsWeg_30000,0.000,1.648,/m/07q2z82,/m/07q2z82,"Accelerating, revving, vroom"
1,3qDzHyrsWeg_30000,0.000,4.497,/m/03m9d0z,/m/03m9d0z,Wind
2,3qDzHyrsWeg_30000,0.000,4.497,/m/0838f,/m/0838f,Water
3,3qDzHyrsWeg_30000,0.000,4.511,/m/02rlv9,/m/02rlv9,"Motorboat, speedboat"
4,3qDzHyrsWeg_30000,4.623,10.000,/m/02rlv9,/m/02rlv9,"Motorboat, speedboat"
5,3qDzHyrsWeg_30000,4.623,10.000,/m/03m9d0z,/m/03m9d0z,Wind
6,3qDzHyrsWeg_30000,4.623,10.000,/m/0838f,/m/0838f,Water
7,3qDzHyrsWeg_30000,4.623,10.000,/m/07q2z82,/m/07q2z82,"Accelerating, revving, vroom"


**Embed Audio**

In [21]:
audio.shape

torch.Size([2, 441344])

In [22]:
labels_df

,segment_id,start_time_seconds,end_time_seconds,label,id,name
0,3qDzHyrsWeg_30000,0.000,1.648,/m/07q2z82,/m/07q2z82,"Accelerating, revving, vroom"
1,3qDzHyrsWeg_30000,0.000,4.497,/m/03m9d0z,/m/03m9d0z,Wind
2,3qDzHyrsWeg_30000,0.000,4.497,/m/0838f,/m/0838f,Water
3,3qDzHyrsWeg_30000,0.000,4.511,/m/02rlv9,/m/02rlv9,"Motorboat, speedboat"
4,3qDzHyrsWeg_30000,4.623,10.000,/m/02rlv9,/m/02rlv9,"Motorboat, speedboat"
5,3qDzHyrsWeg_30000,4.623,10.000,/m/03m9d0z,/m/03m9d0z,Wind
6,3qDzHyrsWeg_30000,4.623,10.000,/m/0838f,/m/0838f,Water
7,3qDzHyrsWeg_30000,4.623,10.000,/m/07q2z82,/m/07q2z82,"Accelerating, revving, vroom"


In [29]:
import torch
import torchaudio
from torchvision.transforms import Normalize
from pytorchvideo.data.clip_sampling import ConstantClipsPerVideoSampler

from imagebind.data import get_clip_timepoints, waveform2melspec

def load_and_transform_audio_data(
    audio_tensors,
    sample_rates,
    device,
    num_mel_bins=128,
    target_length=204,
    desired_sample_rate=16000,
    clip_duration=2,
    clips_per_video=3,
    mean=-4.268,
    std=9.138,
):
    if audio_tensors is None:
        return None

    audio_outputs = []
    clip_sampler = ConstantClipsPerVideoSampler(
        clip_duration=clip_duration, clips_per_video=clips_per_video
    )

    for waveform, sr in zip(audio_tensors, sample_rates):
        if desired_sample_rate != sr:
            waveform = torchaudio.functional.resample(
                waveform, orig_freq=sr, new_freq=desired_sample_rate
            )
        
        all_clips_timepoints = get_clip_timepoints(
            clip_sampler, waveform.size(1) / desired_sample_rate
        )
        all_clips = []
        for clip_timepoints in all_clips_timepoints:
            waveform_clip = waveform[
                :,
                int(clip_timepoints[0] * desired_sample_rate) : int(
                    clip_timepoints[1] * desired_sample_rate
                ),
            ]
            waveform_melspec = waveform2melspec(
                waveform_clip, desired_sample_rate, num_mel_bins, target_length
            )
            all_clips.append(waveform_melspec)

        normalize = Normalize(mean=mean, std=std)
        all_clips = [normalize(ac).to(device) for ac in all_clips]

        all_clips = torch.stack(all_clips, dim=0)
        audio_outputs.append(all_clips)

    return torch.stack(audio_outputs, dim=0)

In [ ]:
# Read a bunch
audio_tensors = []
sample_rates = []

for index in range(1000,2000):
    _, audio, _, info = dataset[index]

    audio_tensors.append(audio)  # list of tensors
    sample_rates.append(audio_fps) # list of corresponding sample rates

In [ ]:
transformed_audio = load_and_transform_audio_data(
    audio_tensors, sample_rates, device='cuda'
)

In [ ]:
transformed_audio.shape

In [32]:
inputs = {
    ModalityType.AUDIO: transformed_audio
}

with torch.no_grad():
    outputs = model(inputs)

audio_embeddings = outputs['audio']

In [34]:
audio_embeddings.shape

torch.Size([1, 1024])